**medium**
ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=1024, n_audio_head=16, n_audio_layer=24, n_vocab=51865, n_text_ctx=448, n_text_state=1024, n_text_head=16, n_text_layer=24)

- tokens: torch.Size([5, 1, 1024])
- audio_features: torch.Size([5, 1500, 1024])
- offset: 0 ++
- positional_embedding: torch.Size([1500, 1024])
- kv_cache: key:Linear(in_features=1024, out_features=1024, bias=False)  value:

# No kv_cache

In [2]:
import whisper
import torch
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "medium"
model = whisper.load_model(model_name).to(device)

In [5]:
n_mels=80
n_audio_ctx=1500

use_folder = os.path.join("/home/mgtv/test_whisper/onnx", model_name)
os.makedirs(use_folder, exist_ok=True)

In [5]:
# encoder input: (batch_size, n_mels, n_ctx)
with torch.no_grad():
    torch.onnx.export(
        model.encoder,
        torch.randn(1, n_mels, 3000).to(device),
        os.path.join(use_folder, "whisper_encoder.onnx"),
        opset_version=13,
        input_names=['feats'],
        output_names=['encoder_out'],
        dynamic_axes = {
        "feats" : {
            0: "batch_size",
        },
        "encoder_out" : {
            0: "batch_size",
        }}
    )

/root/miniconda3/lib/python3.8/site-packages/whisper/model.py:156: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert x.shape[1:] == self.positional_embedding.shape, "incorrect audio shape"


In [3]:
# 没有kv_cache的版本
# tokens = torch.tensor([[50258, 50258, 50258, 50258]]).to(device)
# tokens = torch.randn(1, 1).long().to(device)
tokens = torch.tensor([[50258]]).to(device)
audio_feats = torch.randn(1, 1500, 1024).to(device)
# cache_size = len(model.decoder.blocks) * 2
# kv_cache = torch.randn(cache_size, 1, 4, 1024).to(device)
# offset = torch.tensor(0).to(device)
# kv_cache[:, :, -1] = 0
a = model.decoder(tokens, audio_feats)
print(a.shape)

torch.Size([1, 1, 51865])


In [6]:
with torch.no_grad():
    torch.onnx.export(
    model.decoder,
    (tokens, audio_feats),
    os.path.join(use_folder, "whisper_decoder.onnx"),
    verbose=False,
    opset_version=13,
    input_names = ["tokens", "audio_feats"],
    output_names = ["logits"],
    dynamic_axes = {
        "tokens" : {
            0: "batch_size",
            1: "length",
        },
        "audio_feats" : {
            0: "batch_size",
        },
        
        "logits" : {
            0: "batch_size",
        }
    }
)

In [12]:
## 检测模型正确性 模型大于2G无法检测 
import onnx 

onnx_model = onnx.load("/home/mgtv/test_whisper/onnx/medium/whisper_decoder.onnx") 
onnx.checker.check_model(onnx_model) 

# try: 
#     onnx.checker.check_model(onnx_model) 
# except Exception: 
#     print("Model incorrect") 
# else: 
#     print("Model correct")

RuntimeError: basic_filebuf::underflow error reading the file: Is a directory

## Evaluation No kv_cache
使用命令行需要设置fp16=False

In [1]:
import whisper
import numpy as np
import os
import onnxruntime as ort

model = whisper.load_model("medium")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("/home/mgtv/test_whisper/test1.mp4")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
# _, probs = model.detect_language(mel)
# print(f"Detected language: {max(probs, key=probs.get)}")

In [3]:
# print(model.decoder)
# print(model.decode)

# print(model.onnx_encoder.run(['encoder_out'], {'feats': mel.cpu().unsqueeze(0).numpy()}))
feats = model.encoder(mel.unsqueeze(0))
print(feats)
print(feats[0].dtype)
print(feats[0].shape[0])

<bound method Whisper.encoder of Whisper()>
<bound method Whisper.decoder of Whisper()>
<bound method decode of Whisper()>
True
here encoder
tensor([[[-0.6236,  0.4615, -0.4376,  ..., -0.0392, -1.2181, -0.2081],
         [ 0.9934, -2.4017,  0.9676,  ..., -0.8667, -0.0864, -2.1745],
         [ 0.5403, -1.7654,  0.2960,  ..., -1.4299, -0.4214, -3.0424],
         ...,
         [ 2.0174, -1.1502,  1.1810,  ...,  0.4939,  0.1761, -0.2634],
         [ 1.1018, -0.4732,  1.1714,  ...,  0.3203,  0.6978,  0.1125],
         [-1.3277,  0.8262,  1.6095,  ...,  1.6831, -0.4086,  0.2371]]])
torch.float32
1500


In [3]:
# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result)
print(result.text)

here encoder
shape of run get audio_features:********************
torch.Size([1, 1500, 1024])
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here decoder
here deco

In [1]:
import whisper

model = whisper.load_model("medium")

In [2]:
result = model.transcribe("/home/mgtv/test_whisper/test1.mp4")
print(result)

torch.float32
False
shape of decoding.run() audio_features:********************
torch.Size([1, 1500, 1024])
torch.float32
False
shape of decoding.run() audio_features:********************
torch.Size([1, 1500, 1024])
torch.float32
False
shape of decoding.run() audio_features:********************
torch.Size([1, 1500, 1024])
torch.float32
False
shape of decoding.run() audio_features:********************
torch.Size([1, 1500, 1024])
process spent 21.57512140274048 s. 

decode spent 21.56056809425354 s. 

{'text': '本身我会 | 但是下赛道是第一次 | 那个很帅 | 很刺激 | 我也想要刺激 | 太厉害了 | 2018年演了一部音乐剧 | 那部音乐剧是 | 韩瘋老师跟田庆星导演 | 做的一部音乐剧 | 然后那部音乐剧是让我 | 就是两个月跟大家待在一起 | 然后我那个剧演的是 | 爱情的主题 | 所以就是跟那个男主角 | 就是每天就是 | 就会就靠得很近 | 然后因为我们 | 就是那个剧里面 | 心都亲了十次 | 然后我就觉得通过那部剧 | 就是让我心整个一下就 | 打开了一下 | 打开了 | 就跟之前就完全不一样 | 对 改变了我的性格是不是 | 所以2018年的这个 | 知否知否也是对于我来说 | 非常重要的一首歌 | 那我们来看看 | 这封信是谁写的吧 | 可为你好 | 这三个月的时光音乐之旅 | 让我们彼此都非常幸福', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 0.8, 'text': '本身我会', 'tokens': [8802, 19847, 1654, 12949], 'temper

In [1]:
import whisper

model = whisper.load_model("medium")
result = model.transcribe("/home/mgtv/test_whisper/test1.mp4")
print(result["text"])

load original checkpoint
process spent 9.733283042907715 s. 

decode spent 9.724674224853516 s. 

本身我会但是下赛道是第一次那个很帅很刺激我也想要刺激太厉害了2018年演了一部音乐剧那部音乐剧是韩瘋老师跟田庆星导演做的一部音乐剧然后那部音乐剧是让我就是两个月跟大家待在一起然后我那个剧演的是爱情的主题所以就是跟那个男主角就是每天就是就会就靠得很近然后因为我们就是那个剧里面心都亲了十次然后我就觉得通过那部剧就是让我心整个一下就打开了一下打开了就跟之前就完全不一样对 改变了我的性格是不是所以2018年的这个知否知否也是对于我来说非常重要的一首歌那我们来看看这封信是谁写的吧可为你好这三个月的时光音乐之旅让我们彼此都非常幸福


# With kv_cache

In [1]:
import whisper
import torch
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "medium"
model = whisper.load_model(model_name).to(device)

In [2]:
print(model.device)

cuda


In [ ]:
result = model.transcribe("/home/mgtv/test_whisper/test1.mp4")
print(result["text"])

torch.float16
True
shape of decoding.run() audio_features:********************
torch.Size([1, 1500, 1024])


In [2]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("/home/mgtv/test_whisper/test1.mp4")
print(1)
audio = whisper.pad_or_trim(audio)
print(2)
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)
print(3)

1
2
3


In [ ]:
# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result)
print(result.text)

torch.float16
True
shape of decoding.run() audio_features:********************
torch.Size([1, 1500, 1024])


In [4]:
cache_size = len(model.decoder.blocks) * 2
tokens = torch.tensor([[50258, 50258, 50258, 50258]]).to(device)
audio_feats = torch.randn(1, 1500, 1024).to(device)
kv_cache = torch.randn(cache_size, 1, 4, 1024).to(device)
offset = torch.tensor(0).to(device)
# kv_cache[:, :, -1] = 0
a, new_cache = model.decoder(tokens, audio_feats, kv_cache)
print(a.shape)
print(new_cache.shape)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [4]:
tokens = torch.tensor([[50258]]).to(device)
audio_feats = torch.randn(1, 1500, 1024).to(device)
cache_size = len(model.decoder.blocks) * 2
kv_cache = torch.randn(cache_size, 1, 5, 1024).to(device)
offset = torch.tensor(4).to(device)
a = model.decoder(tokens, audio_feats, kv_cache)
print(a.shape)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
with torch.no_grad():
    torch.onnx.export(
    this_model.decoder,
    (tokens, audio_feats, kv_cache, offset),
    os.path.join(use_folder, "whisper_decoder.onnx"),
    verbose=False,
    opset_version=14,
    input_names = ["tokens", "audio_feats"],
    output_names = ["logits"],
    dynamic_axes = {
        "tokens" : {
            0: "batch_size",
            1: "length",
        },
        "audio_feats" : {
            0: "batch_size",
        },
        "kv_cache" : [1, 2],
        "logits" : {
            0: "batch_size",
        },
        "output_kv_cache" : [1, 2],
    }
)

### Some Ref

In [ ]:
# 知乎超分示例
import onnxruntime 
 
ort_session = onnxruntime.InferenceSession("srcnn.onnx") 
ort_inputs = {'input': input_img} 
ort_output = ort_session.run(['output'], ort_inputs)[0] 
 
ort_output = np.squeeze(ort_output, 0) 
ort_output = np.clip(ort_output, 0, 255) 
ort_output = np.transpose(ort_output, [1, 2, 0]).astype(np.uint8) 
cv2.imwrite("face_ort.png", ort_output)

In [ ]:
import numpy as np

import os
import onnxruntime as ort

providers = ["CUDAExecutionProvider"]  默认CPUExecutionProvider
TensorrtExecutionProvider

In [ ]:
class WhisperOnnx:
    SAMPLERATE = 16000
    MAX_DURATION = 30
    N_FFT = 400
    N_MELS = 80
    HOP_LENGTH = 160
    MAX_FRAMES = 3000

    def __init__(
        self,
        model_dir = None,
        providers = ["CUDAExecutionProvider"],
    ): 
        self.encoder = ort.InferenceSession(
            os.path.join(model_dir, 'whisper_encoder.onnx'),
            providers = providers,
        )
        self.decoder = ort.InferenceSession(
            os.path.join(model_dir, 'whisper_decoder.onnx'),
            providers = providers,
        )
        
        self.max_samples = self.SAMPLERATE * self.MAX_DURATION
        self.tokenizer = get_tokenizer(False, language="en", task="transcribe")
    
ort_inputs = {'input': input_img} 
ort_output = ort_session.run(['output'], ort_inputs)[0]
ws2t = WhisperOnnx(model_dir="/home/mgtv/test_whisper/onnx/medium")

In [4]:
print(3 << 2)

12


## Evaluation of ONNX model

In [ ]:
import librosa
import numpy as np

import os
import onnxruntime as ort
from hyperpyyaml import load_hyperpyyaml

In [ ]:
from typing import Optional
from typing import Tuple
import numpy as np


class Stft:
    """STFT module.
    """

    def __init__(
        self,
        n_fft=400,
        win_length=400,
        hop_length=160,
        center=True,
        window="hann",
        onesided = True,
        normalized = False,
        pad_mode = "reflect",
    ):
        self.stft_kwargs = dict(
            n_fft=n_fft,
            win_length=win_length,
            hop_length=hop_length,
            center=center,
            window=window,
            pad_mode=pad_mode,
        )
        self.onesided = onesided
        self.normalized = normalized

    def __call__(
        self, input: np.ndarray, ilens: np.ndarray = None
    ) -> Tuple[np.ndarray, Optional[np.ndarray]]:
        """STFT forward function.
        Args:
            input: (Batch, Nsamples)
            ilens: (Batch)
        Returns:
            output: (Batch, Frames, Freq, 2)
        """
        output = []
        # iterate over istances in a batch
        for i, instance in enumerate(input):
            stft = librosa.stft(input[i], **self.stft_kwargs)
            output.append(np.array(np.stack([stft.real, stft.imag], -1)))
        output = np.vstack(output).reshape(len(output), *output[0].shape)

        if not self.onesided:
            len_conj = self.n_fft - output.shape[1]
            conj = output[:, 1: 1 + len_conj].flip(1)
            conj[:, :, :, -1].data *= -1
            output = np.concatenate([output, conj], 1)
        if self.normalized:
            output = output * (self.stft_kwargs["window"].shape[0] ** (-0.5))

        # output: (Batch, Freq, Frames, 2=real_imag)
        # -> (Batch, Frames, Freq, 2=real_imag)
        output = output.transpose(0, 2, 1, 3)
        olens = None
        
        # create complex array
        output = output[..., 0] + output[..., 1] * 1j
        return output

In [ ]:
class LogMel:
    """Convert STFT to fbank feats
    The arguments is same as librosa.filters.mel
    Args:
        config.fs: number > 0 [scalar] sampling rate of the incoming signal
        config.n_fft: int > 0 [scalar] number of FFT components
        config.n_mels: int > 0 [scalar] number of Mel bands to generate
        config.fmin: float >= 0 [scalar] lowest frequency (in Hz)
        config.fmax: float >= 0 [scalar] highest frequency (in Hz).
            If `None`, use `fmax = fs / 2.0`
        config.htk: use HTK formula instead of Slaney
    """

    def __init__(
        self,
        fmin = 0,
        fmax = 8000,
        sr = 16000,
        n_fft = 400,
        n_mels = 80,
        htk = False,
        log_base = 10.0,
    ):
        fmin = 0 if fmin is None else fmin
        fmax = sr / 2 if fmax is None else fmax
        _mel_options = dict(
            sr=sr,
            n_fft=n_fft,
            n_mels=n_mels,
            fmin=fmin,
            fmax=fmax,
            htk=htk,
        )
        self.mel_options = _mel_options
        self.log_base = log_base
        melmat = librosa.filters.mel(**_mel_options)
        self.melmat = melmat.T

    def extra_repr(self):
        return ", ".join(f"{k}={v}" for k, v in self.mel_options.items())

    def __call__(
        self,
        feat: np.ndarray,
        ilens: np.ndarray = None,
    ) -> Tuple[np.ndarray, np.ndarray]:
        # feat: (B, T, D1) x melmat: (D1, D2) -> mel_feat: (B, T, D2)
        mel_feat = np.matmul(feat, self.melmat)
        mel_feat = np.clip(mel_feat, 1e-10, float('inf'))

        if self.log_base is None:
            logmel_feat = np.log(mel_feat)
        elif self.log_base == 2.0:
            logmel_feat = np.log2(mel_feat)
        elif self.log_base == 10.0:
            logmel_feat = np.log10(mel_feat)
        else:
            logmel_feat = np.log(mel_feat) / np.log(self.log_base)
        
        # Zero padding
        return logmel_feat

In [ ]:
class WhisperSpeech2Text:
    SAMPLERATE = 16000
    MAX_DURATION = 30
    N_FFT = 400
    N_MELS = 80
    HOP_LENGTH = 160
    MAX_FRAMES = 3000

    def __init__(
        self,
        tag_name: str = None,
        model_dir = None,
        providers = ["CPUExecutionProvider"],
        use_quantized: bool = False,
    ):
        sess_options = ort.SessionOptions()
        sess_options.inter_op_num_threads = 1
        sess_options.intra_op_num_threads = 1

        overrides = dict(
            root_dir=model_dir
        )
        with open(os.path.join(model_dir, "config.yaml"), encoding="utf-8") as reader:
            config = load_hyperpyyaml(reader, overrides)
        
        self._config = config
        self.encoder = ort.InferenceSession(
            config["encoder"]["model_path"],
            providers = providers,
            sess_options = sess_options
        )
        self.decoder = ort.InferenceSession(
            config["decoder"]["model_path"],
            providers = providers,
            sess_options = sess_options
        )
        self.max_samples = self.SAMPLERATE * self.MAX_DURATION
        self.stft = Stft()
        self.logmel = LogMel()
        self.tokenizer = get_tokenizer(False, language="en", task="transcribe")
    
    def __call__(self, xs_tensor: np.array):
        if xs_tensor.ndim > 2:
            raise ValueError("Only single channels is allowed.")
        if xs_tensor.shape[0] > self.max_samples:
            xs_tensor = xs_tensor[:self.max_samples]
        elif xs_tensor.shape[0] < self.max_samples:
            pads = self.max_samples - xs_tensor.shape[0]
            xs_tensor = np.pad(xs_tensor, (0, pads), 'constant')
        xs_tensor = self.get_feats(xs_tensor[None])
        hs_tensor, = self.encoder.run(["output"], {"input" : xs_tensor})
        return hs_tensor

    def get_feats(self, xs_tensor: np.array):
        xs_tensor = self.stft(xs_tensor)[:, :self.MAX_FRAMES]
        xs_tensor = xs_tensor.real ** 2 + xs_tensor.imag ** 2
        xs_tensor = self.logmel(xs_tensor)
        # Whisper Normalization:
        amax = np.amax(xs_tensor) - 8.0
        xs_tensor = np.clip(xs_tensor, None, amax)
        xs_tensor = (xs_tensor + 4.0) / 4.0
        return np.swapaxes(xs_tensor, 1, 2)

In [ ]:
ws2t = WhisperSpeech2Text(model_dir="../build/tiny")

print(ws2t._config)

In [ ]:
try:
    signal = np.random.randn(16000 * 30).astype(np.float32)
    hs_tensor = ws2t(signal)
    # start_id = np.array([[50258, 5, 6, 2, 6, 1, 5, 1, 6]], dtype=int)
    # ys_tensor, = ws2t.decoder.run(["logits"], {"input_text" : start_id, "input_speech": hs_tensor})
    # print(ys_tensor)
    print(hs_tensor.shape)
except Exception as e:
    print(e)
    pass

In [ ]:
tokens = ws2t.tokenizer.encode("<|endoftext|>I would like to know about how this is workidns<|endoftext|>")
print(tokens)
text = ws2t.tokenizer.decode(tokens)
print(text)